# NutriAI Health Coach
Visualización interactiva de recomendaciones nutricionales personalizadas.

In [1]:
# Inicialización de librerías y ontología
import streamlit as st
from rdflib import Graph, Namespace, RDF
import pandas as pd

# Crear grafo y namespaces
g = Graph()
FOOD = Namespace("http://example.org/food/")
NUT = Namespace("http://example.org/nutrient/")
COND = Namespace("http://example.org/condition/")
EX = Namespace("http://example.org/ex/")

g.bind("food", FOOD)
g.bind("nut", NUT)
g.bind("cond", COND)
g.bind("ex", EX)

# Alimentos y nutrientes
g.add((FOOD.Tofu, RDF.type, FOOD.PlantBasedProtein))
g.add((FOOD.Tofu, NUT.hasNutrient, NUT.Calcium))
g.add((FOOD.Tofu, NUT.hasNutrient, NUT.Protein))
g.add((FOOD.Broccoli, RDF.type, FOOD.Vegetable))
g.add((FOOD.Broccoli, NUT.hasNutrient, NUT.Fiber))
g.add((FOOD.Broccoli, NUT.hasNutrient, NUT.Calcium))
g.add((FOOD.Sardines, RDF.type, FOOD.Fish))
g.add((FOOD.Sardines, NUT.hasNutrient, NUT.Calcium))
g.add((FOOD.Sardines, NUT.hasNutrient, NUT.Protein))
g.add((FOOD.Lentils, RDF.type, FOOD.Legume))
g.add((FOOD.Lentils, NUT.hasNutrient, NUT.Fiber))
g.add((FOOD.Lentils, NUT.hasNutrient, NUT.Protein))
g.add((FOOD.Oats, RDF.type, FOOD.Grain))
g.add((FOOD.Oats, NUT.hasNutrient, NUT.Fiber))
g.add((FOOD.Spinach, RDF.type, FOOD.Vegetable))
g.add((FOOD.Spinach, NUT.hasNutrient, NUT.Calcium))
g.add((FOOD.Spinach, NUT.hasNutrient, NUT.Iron))
g.add((FOOD.Milk, RDF.type, FOOD.Dairy))
g.add((FOOD.Milk, NUT.hasNutrient, NUT.Calcium))
g.add((FOOD.Milk, NUT.hasNutrient, NUT.VitaminD))

# Beneficios
g.add((NUT.Calcium, EX.beneficialFor, COND.Hypertension))
g.add((NUT.Calcium, EX.beneficialFor, COND.Osteoporosis))
g.add((NUT.Fiber, EX.beneficialFor, COND.Diabetes))
g.add((NUT.Protein, EX.beneficialFor, COND.MuscleLoss))
g.add((NUT.Iron, EX.beneficialFor, COND.Anemia))
g.add((NUT.VitaminD, EX.beneficialFor, COND.Osteoporosis))

# Tiempos estimados de mejora
tiempo_mejoras = {
    "glucosa": "3 semanas",
    "presion_arterial": "2-4 semanas",
    "colesterol_LDL": "4 semanas",
    "hemoglobina": "6 semanas",
    "densidad_osea": "6-12 meses"
}

In [2]:
# Inferencias y recomendaciones
def inferir_alimentos_para_condicion(condicion_uri):
    nutrientes_beneficiosos = set()
    for s, p, o in g.triples((None, EX.beneficialFor, condicion_uri)):
        nutrientes_beneficiosos.add(s)
    alimentos = []
    for alimento, _, nutriente in g.triples((None, NUT.hasNutrient, None)):
        if nutriente in nutrientes_beneficiosos:
            alimentos.append(str(alimento).split("/")[-1])
    return sorted(set(alimentos))

def recomendar_por_perfil_clinico(glucosa, presion, anemia, osteoporosis):
    recomendaciones = []
    if glucosa > 110:
        recomendaciones.append("Consumir alimentos con fibra soluble como el brócoli, avena o lentejas. Mejora estimada en {}.".format(tiempo_mejoras["glucosa"]))
    if presion > 140:
        recomendaciones.append("Incluir alimentos ricos en calcio como el tofu, sardinas, leche o espinacas. Mejora estimada en {}.".format(tiempo_mejoras["presion_arterial"]))
    if anemia:
        recomendaciones.append("Aumentar alimentos ricos en hierro como espinacas. Mejora estimada en {}.".format(tiempo_mejoras["hemoglobina"]))
    if osteoporosis:
        recomendaciones.append("Consumir alimentos con calcio y vitamina D como leche y tofu. Mejora estructural estimada en {}.".format(tiempo_mejoras["densidad_osea"]))
    return recomendaciones

In [3]:
# Simulación
glucosa = 115
presion = 145
anemia = True
osteoporosis = True

recomendaciones = recomendar_por_perfil_clinico(glucosa, presion, anemia, osteoporosis)
for r in recomendaciones:
    print("-", r)

df_hipertension = pd.DataFrame(inferir_alimentos_para_condicion(COND.Hypertension), columns=["Alimentos para Hipertensión"])
df_hipertension

- Consumir alimentos con fibra soluble como el brócoli, avena o lentejas. Mejora estimada en 3 semanas.
- Incluir alimentos ricos en calcio como el tofu, sardinas, leche o espinacas. Mejora estimada en 2-4 semanas.
- Aumentar alimentos ricos en hierro como espinacas. Mejora estimada en 6 semanas.
- Consumir alimentos con calcio y vitamina D como leche y tofu. Mejora estructural estimada en 6-12 meses.


,Alimentos para Hipertensión
0,Broccoli
1,Milk
2,Sardines
3,Spinach
4,Tofu
